In [7]:
import pandas as pd
import numpy as np
import sys

import warnings
import math

df=pd.read_csv('../data/data.csv.gz', compression='gzip', sep=',')
df['timestamp'] = pd.to_datetime(df['time_stamp'], format="%Y-%m-%d %H:%M:%S").dt.tz_localize('CET')

In [8]:
stock_delta = df['stock_id'] - df['stock_id'].shift()
time_delta = (df['timestamp'] - df['timestamp'].shift()).fillna(10000).dt.total_seconds()

In [9]:
stock_split_index=df.index[stock_delta!=0].tolist()
stock_split_index.append(len(df))

In [10]:
day_split_index=df.index[abs(time_delta)>36000].tolist()
day_split_index.insert(0, 0)
day_split_index.append(len(df))

In [11]:
stock_id_list = df['stock_id'].unique().tolist()
i = 0
for stock_index in stock_id_list:
    
    print("index: {}, id:{}".format(i, stock_index))
    i+=1

index: 0, id:101
index: 1, id:1027
index: 2, id:120
index: 3, id:139301
index: 4, id:15285
index: 5, id:160271
index: 6, id:161
index: 7, id:18634
index: 8, id:281
index: 9, id:283
index: 10, id:285
index: 11, id:300
index: 12, id:323
index: 13, id:340
index: 14, id:3524
index: 15, id:361
index: 16, id:366
index: 17, id:3966
index: 18, id:401
index: 19, id:402
index: 20, id:45
index: 21, id:46
index: 22, id:47
index: 23, id:4928
index: 24, id:5095
index: 25, id:81
index: 26, id:812
index: 27, id:819
index: 28, id:992
index: 29, id:999


In [12]:
current_stock_index = 0
time_series_list = []
time_series_all_list = []
for i in range(len(day_split_index) - 1):
    start_index = day_split_index[i]
    end_index = day_split_index[i + 1]
    day_time_series = df.iloc[start_index:end_index]
    time_series_list.append(day_time_series)
    if end_index == stock_split_index[current_stock_index + 1]:
        print("finished stock: {}, got {} time series".format(current_stock_index, len(time_series_list)))
        time_series_all_list.append(time_series_list)
        time_series_list = []
        current_stock_index += 1

finished stock: 0, got 61 time series
finished stock: 1, got 61 time series
finished stock: 2, got 61 time series
finished stock: 3, got 61 time series
finished stock: 4, got 61 time series
finished stock: 5, got 61 time series
finished stock: 6, got 61 time series
finished stock: 7, got 61 time series
finished stock: 8, got 61 time series
finished stock: 9, got 61 time series
finished stock: 10, got 61 time series
finished stock: 11, got 61 time series
finished stock: 12, got 61 time series
finished stock: 13, got 61 time series
finished stock: 14, got 61 time series
finished stock: 15, got 61 time series
finished stock: 16, got 61 time series
finished stock: 17, got 61 time series
finished stock: 18, got 61 time series
finished stock: 19, got 61 time series
finished stock: 20, got 61 time series
finished stock: 21, got 61 time series
finished stock: 22, got 61 time series
finished stock: 23, got 61 time series
finished stock: 24, got 61 time series
finished stock: 25, got 61 time ser

In [13]:
for stock_id in range(len(time_series_all_list)):
    print("handling stock_id: {}".format(stock_id))
    for day_id in range(len(time_series_all_list[stock_id])):
        # :-1 is that we don't like the last record at 17:30 which is a aggregated number.
        df = time_series_all_list[stock_id][day_id]
        # some data might miss, we must make a right join with full time series
        # and do fillna.
        df2 = df.set_index('timestamp')
        ts = df2.index.min()
        start_time_str = "{}-{:02d}-{:02d} 8:54:00".format(ts.year, ts.month, ts.day)
        start_ts = pd.Timestamp(start_time_str, tz=ts.tz)
        # periods=510 means from 9 to 17.29
        dti = pd.date_range(start_ts , periods=516, freq='min').to_series(keep_tz=True).rename('time')
        # remove from 17.25 - 17.28
        #dti.drop(dti.tail(5).head(4).index, inplace=True)
        df3 = df2.join(dti, how='right')
        if day_id == 0: # the first day, we must set the value from 8.55-8.59 as same as 9.00
            df3['last'].iloc[0] = df3['last'].iloc[5]
        else:
            df3['last'].iloc[0] = time_series_all_list[stock_id][day_id-1]['last'].iloc[-1]
        
        
        df3['last'].interpolate(method='linear', inplace=True)
        df3['volume'].iloc[:6] = df3['volume'].iloc[6] / 6
        df3['volume'].iloc[6] = df3['volume'].iloc[6] / 6
        df3['volume'].iloc[-5:] = df3['volume'].iloc[-1]/5
        
        
        df = df3.reset_index().rename({'index':'timestamp'}, axis=1)
        #df['timestamp'] = pd.to_datetime(df['time_stamp'], format="%Y-%m-%d %H:%M:%S").dt.tz_convert('Europe/Stockholm')
        df['ema_10'] = df['last'].ewm(span=10, adjust=False).mean()
        df['ema_20'] = df['last'].ewm(span=20, adjust=False).mean()
        df['diff_ema_10']=(df['ema_10'].diff()[1:]/df['ema_10']).fillna(0)
        # the first diff at 9:00 is the difference between today's open and yesterday's last.

        df['diff_ema_20']=(df['ema_20'].diff()[1:]/df['ema_20']).fillna(0)
        df['value_ema_10_beta_99'] = 0
        df['value_ema_20_beta_99'] = 0
        df['value_ema_10_beta_98'] = 0
        df['value_ema_20_beta_98'] = 0
        for iter_id in range(10):
            df['value_ema_10_beta_99'] = df['diff_ema_10'].shift(-1).fillna(0) + \
                0.99 * df['value_ema_10_beta_99'].shift(-1).fillna(0)
            df['value_ema_10_beta_98'] = df['diff_ema_10'].shift(-1).fillna(0) + \
                0.98 * df['value_ema_10_beta_98'].shift(-1).fillna(0)
            df['value_ema_20_beta_99'] = df['diff_ema_20'].shift(-1).fillna(0) + \
                0.99 * df['value_ema_20_beta_99'].shift(-1).fillna(0)
            df['value_ema_20_beta_98'] = df['diff_ema_20'].shift(-1).fillna(0) + \
                0.98 * df['value_ema_20_beta_98'].shift(-1).fillna(0)
        # drop the first row because diff is nan    
        #df.drop(0, inplace=True)
        time_series_all_list[stock_id][day_id] = df

handling stock_id: 0


/home/ken/.local/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


handling stock_id: 1
handling stock_id: 2
handling stock_id: 3
handling stock_id: 4
handling stock_id: 5
handling stock_id: 6
handling stock_id: 7
handling stock_id: 8
handling stock_id: 9
handling stock_id: 10
handling stock_id: 11
handling stock_id: 12
handling stock_id: 13
handling stock_id: 14
handling stock_id: 15
handling stock_id: 16
handling stock_id: 17
handling stock_id: 18
handling stock_id: 19
handling stock_id: 20
handling stock_id: 21
handling stock_id: 22
handling stock_id: 23
handling stock_id: 24
handling stock_id: 25
handling stock_id: 26
handling stock_id: 27
handling stock_id: 28
handling stock_id: 29


In [14]:
time_series_all_list[0][2]

,timestamp,stock_id,time_stamp,open,high,low,last,volume,time,ema_10,ema_20,diff_ema_10,diff_ema_20,value_ema_10_beta_99,value_ema_20_beta_99,value_ema_10_beta_98,value_ema_20_beta_98
0,2019-01-30 08:54:00+01:00,NaN,NaN,NaN,NaN,NaN,80.22,422.0,2019-01-30 08:54:00+01:00,80.220000,80.220000,0.000000e+00,0.000000e+00,-8.173158e-04,-5.681707e-04,-7.865051e-04,-5.443069e-04
1,2019-01-30 08:55:00+01:00,101.0,2019-01-30 08:55:00+00,80.16,80.16,80.16,80.16,422.0,2019-01-30 08:55:00+01:00,80.209091,80.214286,-1.360082e-04,-7.123776e-05,-8.371189e-04,-6.022488e-04,-7.996971e-04,-5.742619e-04
2,2019-01-30 08:56:00+01:00,101.0,2019-01-30 08:56:00+00,80.14,80.14,80.14,80.14,422.0,2019-01-30 08:56:00+01:00,80.196529,80.207211,-1.566400e-04,-8.820691e-05,-7.263023e-04,-5.665690e-04,-6.917294e-04,-5.391759e-04
3,2019-01-30 08:57:00+01:00,101.0,2019-01-30 08:57:00+00,80.16,80.18,80.16,80.16,422.0,2019-01-30 08:57:00+01:00,80.189887,80.202715,-8.282370e-05,-5.606138e-05,-7.647675e-04,-6.019133e-04,-7.260980e-04,-5.716917e-04
4,2019-01-30 08:58:00+01:00,101.0,2019-01-30 08:58:00+00,80.16,80.16,80.16,80.16,422.0,2019-01-30 08:58:00+01:00,80.184453,80.198647,-6.776943e-05,-5.072477e-05,-8.809083e-04,-6.782255e-04,-8.331892e-04,-6.424614e-04
5,2019-01-30 08:59:00+01:00,101.0,2019-01-30 08:59:00+00,80.16,80.16,80.16,80.16,422.0,2019-01-30 08:59:00+01:00,80.180007,80.194966,-5.545079e-05,-4.589595e-05,-1.020012e-03,-7.703486e-04,-9.635662e-04,-7.288779e-04
6,2019-01-30 09:00:00+01:00,101.0,2019-01-30 09:00:00+00,80.20,80.20,80.18,80.18,422.0,2019-01-30 09:00:00+01:00,80.180006,80.193541,-1.633299e-08,-1.777358e-05,-1.099712e-03,-8.358482e-04,-1.046567e-03,-7.946804e-04
7,2019-01-30 09:01:00+01:00,101.0,2019-01-30 09:01:00+00,80.16,80.18,80.14,80.14,9596.0,2019-01-30 09:01:00+01:00,80.172732,80.188441,-9.072659e-05,-6.358903e-05,-1.075974e-03,-8.485295e-04,-1.027184e-03,-8.085025e-04
8,2019-01-30 09:02:00+01:00,101.0,2019-01-30 09:02:00+00,80.14,80.16,80.12,80.12,2724.0,2019-01-30 09:02:00+01:00,80.163144,80.181923,-1.196018e-04,-8.129311e-05,-9.710219e-04,-8.152182e-04,-9.306581e-04,-7.787692e-04
9,2019-01-30 09:03:00+01:00,101.0,2019-01-30 09:03:00+00,80.14,80.14,80.14,80.14,1000.0,2019-01-30 09:03:00+01:00,80.158936,80.177931,-5.249671e-05,-4.979788e-05,-9.733687e-04,-8.312755e-04,-9.376697e-04,-7.968965e-04


In [15]:
columns = ['timestamp','last', 'diff_ema_20', 'value_ema_20_beta_99', 'value_ema_20_beta_98', 
           'diff_ema_10', 'value_ema_10_beta_99', 'value_ema_10_beta_98', 'volume']

print(time_series_all_list[0][1][columns])

                    timestamp   last   diff_ema_20  value_ema_20_beta_99  \
0   2019-01-29 08:54:00+01:00  79.02  0.000000e+00          6.730801e-03   
1   2019-01-29 08:55:00+01:00  79.74  8.670207e-04          6.321640e-03   
2   2019-01-29 08:56:00+01:00  79.88  9.521272e-04          5.806112e-03   
3   2019-01-29 08:57:00+01:00  79.84  8.126655e-04          5.367869e-03   
4   2019-01-29 08:58:00+01:00  79.86  7.587341e-04          4.948726e-03   
5   2019-01-29 08:59:00+01:00  79.90  7.339810e-04          4.500547e-03   
6   2019-01-29 09:00:00+01:00  79.88  6.396631e-04          4.141710e-03   
7   2019-01-29 09:01:00+01:00  79.92  6.263306e-04          3.726007e-03   
8   2019-01-29 09:02:00+01:00  79.92  5.663591e-04          3.306356e-03   
9   2019-01-29 09:03:00+01:00  79.96  5.600295e-04          2.834294e-03   
10  2019-01-29 09:04:00+01:00  79.94  4.825123e-04          2.364588e-03   
11  2019-01-29 09:05:00+01:00  79.94  4.363682e-04          1.959604e-03   
12  2019-01-

In [16]:
%matplotlib notebook
from ipywidgets import *

widgets.IntSlider(min=0,max=len(time_series_all_list[0]) - 1,step=1,value=10)

@interact(stock_id=widgets.IntSlider(min=0,max=len(time_series_all_list) - 1,step=1,value=10), day_id = widgets.IntSlider(min=0,max=len(time_series_all_list[0]) - 1,step=1,value=10))
def plot(stock_id, day_id ):
    df = time_series_all_list[stock_id][day_id]
    df.plot(x='timestamp', y=['last','ema_20'])
    df.plot(x='timestamp', y=['value_ema_20_beta_98'])

In [18]:

from functools import reduce

# to order the columns in the order: last1,last2...last30, diff1, diff2,...diff30, value1, value2,...value30
def func(x):
    if 'timestamp' in x:
        return 0
    num = int(x.split('_')[-1])

    if 'last' in x:
        return 100 + num
    elif 'volume' in x:
        return 150 + num
    elif 'diff_ema_10' in x:
        return 200 + num
    elif 'diff_ema_20' in x:
        return 300 + num
    elif 'value_ema_10_beta_98' in x:
        return 400 + num
    elif 'value_ema_10_beta_99' in x:
        return 500 + num
    elif 'value_ema_20_beta_98' in x:
        return 600 + num
    elif 'value_ema_20_beta_99' in x:
        return 700 + num


def merge(df1, df2):
    global index
    merged = df1.set_index('timestamp').join(df2.set_index('timestamp'), how='outer')
    index += 1

    return merged.reset_index().rename({'index':'timestamp'}, axis=1)


columns_wanted = ['timestamp','last', 'diff_ema_20', 'value_ema_20_beta_99', 'value_ema_20_beta_98', 
           'diff_ema_10', 'value_ema_10_beta_99', 'value_ema_10_beta_98', 'volume']
df_day_list = []

for day_id in range(len(time_series_all_list[0])):
    print("processing day: {}".format(day_id))
    df_list = []
    for i in range(len(stock_id_list)):
        rename_map = {'last': 'last_' + str(i),
                      'diff_ema_20':'diff_ema_20_' + str(i),
                      'value_ema_20_beta_99':'value_ema_20_beta_99_' + str(i),
                      'value_ema_20_beta_98':'value_ema_20_beta_98_' + str(i),
                      'diff_ema_10':'diff_ema_10_' + str(i),
                      'value_ema_10_beta_99':'value_ema_10_beta_99_' + str(i),
                      'value_ema_10_beta_98':'value_ema_10_beta_98_'+ str(i),
                      'volume': 'volume_' + str(i)
                     }
        df = time_series_all_list[i][day_id][columns_wanted].rename(rename_map, axis=1)
        df_list.append(df)

    index = 0
    df_merged_daily = reduce(merge, df_list)
    columns = df_merged_daily.columns.tolist()
    columns.sort(key=func)
    df_merged_sorted = df_merged_daily[columns]
    
    df_day_list.append(df_merged_sorted)
    if day_id == 0:
        df_merged = df_merged_sorted
    else:
        df_merged = df_merged.append(df_merged_sorted)

processing day: 0
processing day: 1
processing day: 2
processing day: 3
processing day: 4
processing day: 5
processing day: 6
processing day: 7
processing day: 8
processing day: 9
processing day: 10
processing day: 11
processing day: 12
processing day: 13
processing day: 14
processing day: 15
processing day: 16
processing day: 17
processing day: 18
processing day: 19
processing day: 20
processing day: 21
processing day: 22
processing day: 23
processing day: 24
processing day: 25
processing day: 26
processing day: 27
processing day: 28
processing day: 29
processing day: 30
processing day: 31
processing day: 32
processing day: 33
processing day: 34
processing day: 35
processing day: 36
processing day: 37
processing day: 38
processing day: 39
processing day: 40
processing day: 41
processing day: 42
processing day: 43
processing day: 44
processing day: 45
processing day: 46
processing day: 47
processing day: 48
processing day: 49
processing day: 50
processing day: 51
processing day: 52
pro

In [19]:

df_merged

,timestamp,last_0,last_1,last_2,last_3,last_4,last_5,last_6,last_7,last_8,...,value_ema_20_beta_99_20,value_ema_20_beta_99_21,value_ema_20_beta_99_22,value_ema_20_beta_99_23,value_ema_20_beta_99_24,value_ema_20_beta_99_25,value_ema_20_beta_99_26,value_ema_20_beta_99_27,value_ema_20_beta_99_28,value_ema_20_beta_99_29
0,2019-01-28 08:54:00+01:00,79.90,115.650000,210.00,NaN,218.200,82.85,392.8,198.15,91.36,...,2.319536e-04,1.930045e-04,7.415716e-04,-3.621122e-04,-3.043401e-04,5.543303e-04,1.057834e-03,2.032367e-04,-1.757292e-05,2.849731e-04
1,2019-01-28 08:55:00+01:00,79.68,115.625000,209.70,224.300000,218.600,82.83,393.0,198.20,91.32,...,2.175307e-04,1.795437e-04,6.928820e-04,9.689196e-05,-4.845580e-04,6.340996e-04,1.055442e-03,6.612824e-04,-1.191211e-04,3.042311e-04
2,2019-01-28 08:56:00+01:00,79.80,115.600000,209.75,224.400000,218.650,82.85,393.2,198.40,91.32,...,1.353014e-04,1.815843e-04,5.772643e-04,2.909518e-04,-6.716628e-04,7.493176e-04,8.751327e-04,8.914813e-04,-2.807747e-04,2.793276e-04
3,2019-01-28 08:57:00+01:00,79.86,115.575000,209.80,224.400000,218.450,82.87,393.0,198.25,91.40,...,9.838809e-05,1.628556e-04,4.132384e-04,6.205463e-04,-7.948322e-04,8.091664e-04,6.560589e-04,1.102722e-03,-3.743543e-04,1.325581e-04
4,2019-01-28 08:58:00+01:00,79.90,115.550000,209.90,224.400000,218.325,82.86,393.0,198.20,91.40,...,1.469927e-04,2.126011e-04,4.166048e-04,8.855022e-04,-8.066100e-04,9.063338e-04,4.013551e-04,1.338156e-03,-4.588839e-04,8.413266e-05
5,2019-01-28 08:59:00+01:00,79.90,115.650000,210.00,224.400000,218.200,82.85,392.8,198.15,91.36,...,1.364242e-04,2.374996e-04,4.405759e-04,7.748833e-04,-9.151293e-04,9.498415e-04,1.528327e-04,1.380922e-03,-5.495154e-04,3.995486e-05
6,2019-01-28 09:00:00+01:00,79.88,115.600000,209.90,224.400000,218.150,82.85,392.8,198.35,91.36,...,1.956179e-05,2.145459e-04,4.039684e-04,6.713085e-04,-1.037147e-03,9.443658e-04,-1.317794e-04,1.275706e-03,-6.601019e-04,3.831828e-05
7,2019-01-28 09:01:00+01:00,79.88,115.550000,210.00,224.350000,218.100,82.85,392.8,198.30,91.32,...,1.608480e-05,1.708101e-04,2.948602e-04,6.044558e-04,-1.237491e-03,9.447248e-04,-3.791339e-04,1.135300e-03,-8.264867e-04,-1.410815e-05
8,2019-01-28 09:02:00+01:00,79.88,115.500000,209.70,224.300000,218.000,82.84,392.7,198.35,91.34,...,1.292273e-05,1.528723e-04,1.193916e-04,5.743246e-04,-1.232414e-03,7.703104e-04,-5.926743e-04,1.029798e-03,-9.262599e-04,-6.202631e-05
9,2019-01-28 09:03:00+01:00,79.92,115.500000,209.60,224.300000,218.150,82.84,392.5,198.40,91.30,...,1.004542e-05,1.584920e-04,-1.168881e-04,5.505753e-04,-1.250919e-03,7.039309e-04,-7.755158e-04,9.147866e-04,-9.652404e-04,-5.900822e-05


In [20]:
columns = df_merged_sorted.columns
last_columns = []
volume_columns = []
diff_ema_10_columns = []
diff_ema_20_columns = []
value_ema_10_beta_98_columns = []
value_ema_10_beta_99_columns = []
value_ema_20_beta_98_columns = []
value_ema_20_beta_99_columns = []
for item in columns:
    if 'last' in item:
        last_columns.append(item)
    elif 'volume' in item:
        volume_columns.append(item)
    elif 'diff_ema_10' in item:
        diff_ema_10_columns.append(item)
    elif 'diff_ema_20' in item:
        diff_ema_20_columns.append(item)
    elif 'value_ema_10_beta_98' in item:
        value_ema_10_beta_98_columns.append(item)
    elif 'value_ema_10_beta_99' in item:
        value_ema_10_beta_99_columns.append(item)
    elif 'value_ema_20_beta_98' in item:
        value_ema_20_beta_98_columns.append(item)
    elif 'value_ema_20_beta_99' in item:
        value_ema_20_beta_99_columns.append(item)



In [21]:
df_merged[['timestamp'] + volume_columns + last_columns + diff_ema_10_columns + value_ema_10_beta_98_columns].to_csv('data-prep-ema10-beta98.csv')
df_merged[['timestamp'] + volume_columns + last_columns + diff_ema_10_columns + value_ema_10_beta_99_columns].to_csv('data-prep-ema10-beta99.csv')
df_merged[['timestamp'] + volume_columns + last_columns + diff_ema_20_columns + value_ema_20_beta_98_columns].to_csv('data-prep-ema20-beta98.csv')
df_merged[['timestamp'] + volume_columns + last_columns + diff_ema_20_columns + value_ema_20_beta_99_columns].to_csv('data-prep-ema20-beta99.csv')